In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional
from tensorflow.keras.optimizers import Adam
import numpy as np
import json
import random
import warnings

warnings.filterwarnings('ignore')

# Load intent configuration (directly load the list without accessing ['intents'])
with open('intent1.json', 'r') as f:
    intents_config = json.load(f)

# Load menu data
with open('mockMenu 1.json', 'r') as f:
    menu_data = json.load(f)

# Data preprocessing
intents = []
unique_intents = []
text_input = []
response_for_intent = {}
query_for_intent = {}

# Iterate over intents_config directly as it is a list
for intent in intents_config:
    intent_name = intent['intent']
    
    if intent_name not in unique_intents:
        unique_intents.append(intent_name)
    
    # Use 'question' key for text input as per your JSON structure
    text_input.append(intent['question'])
    intents.append(intent_name)
    
    # Add responses for each intent
    if intent_name not in response_for_intent:
        response_for_intent[intent_name] = []
    response_for_intent[intent_name].append(intent['response'])
    
    # Add context retrieval queries if any
    if intent_name not in query_for_intent:
        query_for_intent[intent_name] = intent.get('context-retrieval', '')

# Tokenizer setup and text to sequence conversion
tokenizer = Tokenizer(filters='', oov_token='<unk>')
tokenizer.fit_on_texts(text_input)
sequences = tokenizer.texts_to_sequences(text_input)
padded_sequences = pad_sequences(sequences, padding='pre')

# Creating intent to index mapping
intent_to_index = {}
categorical_target = []
index = 0

for intent in intents:
    if intent not in intent_to_index:
        intent_to_index[intent] = index
        index += 1
    categorical_target.append(intent_to_index[intent])

num_classes = len(intent_to_index)
index_to_intent = {index: intent for intent, index in intent_to_index.items()}

# Convert target to categorical vector
categorical_vec = tf.keras.utils.to_categorical(categorical_target, num_classes=num_classes)
categorical_vec = categorical_vec.astype('int32')

# Model parameters
epochs = 100
embed_dim = 300
lstm_num = 50
output_dim = categorical_vec.shape[1]

# Define model
model = Sequential([
    Embedding(len(tokenizer.word_index) + 1, embed_dim),
    Bidirectional(LSTM(lstm_num, dropout=0.1)),
    Dense(lstm_num, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    Dense(output_dim, activation='softmax')
])

# Compile model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train model
model.fit(padded_sequences, categorical_vec, epochs=epochs, verbose=1)

# Function to predict intent based on the input sentence
def predict_intent(sentence):
    sent_tokens = []
    words = sentence.split()
    for word in words:
        if word in tokenizer.word_index:
            sent_tokens.append(tokenizer.word_index[word])
        else:
            sent_tokens.append(tokenizer.word_index.get('<unk>', 0))
    sent_tokens = tf.expand_dims(sent_tokens, 0)
    pred = model(sent_tokens)
    pred_class = np.argmax(pred.numpy(), axis=1)
    intent_name = index_to_intent[pred_class[0]]
    return intent_name

# Function to execute context-retrieval query (code execution)
def execute_query(query_code, query, menu_data):
    # Prepare local variables for execution
    local_vars = {'menu_data': menu_data, 'query': query, 'result': None}

    # Ensure query_code is properly formatted with new lines and indentation
    formatted_query_code = '\n'.join(f'    {line}' if line.strip() else line for line in query_code.split('\n'))
    
    # Define the query function code
    exec_code = f"""
def run_query(menu_data, query):
    result = None
    {formatted_query_code}
    return result
"""
    
    try:
        # Execute the query code
        exec(exec_code, globals(), local_vars)
        # Call the query function and get the result
        result = local_vars['run_query'](menu_data, query)
    except Exception as e:
        print(f"Error executing query: {e}")
        result = "No result found."
    
    return result

# Function to generate a response
def response(sentence):
    intent_name = predict_intent(sentence)
    intent_config = next((i for i in intents_config if i['intent'] == intent_name), None)
    
    if intent_config and 'context-retrieval' in intent_config:
        query_code = intent_config['context-retrieval']
        
        # Execute the query using the query_code
        result = execute_query(query_code, sentence, menu_data)
        return result, intent_name
    else:
        return random.choice(response_for_intent.get(intent_name, ["Sorry, I didn't understand the request."])), intent_name

# Testing loop to interact with the bot
print("Note: Enter 'quit' to break the loop.")
while True:
    query = input('You: ')
    if query.lower() == 'quit':
        break
    bot_response, typ = response(query)
    print(f'Geek: {bot_response} -- TYPE: {typ}')
    print()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.1000 - loss: 2.6375
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2000 - loss: 2.6310
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2000 - loss: 2.6072
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.2500 - loss: 2.5957
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.2500 - loss: 2.5846
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2000 - loss: 2.5634
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3000 - loss: 2.5577
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3500 - loss: 2.5421
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.2000 - loss: 2.5669
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4000 - loss: 2.5079
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3500 - loss: 2.5182
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.4000 - loss